In [22]:
"""
Program Name: Plasma Kinetic Parameter Estimator (PlKinParaEst)
Author: SJBaek
Date: 2024-11-22 ~
Description:
    - This program automatically run ZDPlasKin with optimization module for kinetic parameter estimation
"""

'\nProgram Name: Plasma Kinetic Parameter Estimator (PlKinParaEst)\nAuthor: SJBaek\nDate: 2024-11-22 ~\nDescription:\n    - This program automatically run ZDPlasKin with optimization module for kinetic parameter estimation\n'

In [23]:
# Modules
import subprocess
from IPython.display import clear_output
import pandas as pd
import re
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel
import ast
from scipy.stats import norm


In [24]:
# compile zdplaskin
def run_prep(inp_path):
    try:
        process = subprocess.Popen(
            'preprocessor.exe',
            stdout = subprocess.DEVNULL,        # ignore outputs
            stderr = subprocess.DEVNULL,        # ignore errors
            stdin = subprocess.PIPE,            # recognize input
            universal_newlines=True
        )
        
        process.stdin.write(inp_path)
        process.stdin.flush()                   # send a data

        while process.poll() is None:           # check the program state, if None, program is still in the run
            process.stdin.write('.\n')
            process.stdin.flush()
    except:
        pass
    print('check the run of preprocessor')
    return process

In [25]:
# Compile exe
def compile_zdp(name):
    compile_command = [
        'gfortran', '-o', name, 'dvode_f90_m.F90', 'zdplaskin_m.F90',
        'run_plasRxn_v2.F90', 'bolsig_x86_64_g.dll'
    ]
    
    try:
        subprocess.run(compile_command)
    except:
        pass
    print('check the compiler')

In [26]:
# Run exe
def run_exe(exe_path):
    try:
        process = subprocess.Popen(
            exe_path,
            stdout = subprocess.PIPE, # Read standard outputs
            stderr = subprocess.PIPE, # Read standard errors
            universal_newlines=True,  # outputs to str variables
            bufsize = 1               # control the size of buffer
        )

        log_flag = False             # The flag for starting log after "Caculation Start!!"
        while True:
            output = process.stdout.readline()
            if not output:
                break
            if "Calculation Start" in output:
                log_flag = True

            if log_flag:
                print(f'\r{output.strip()}           ',end='',flush=True)

            if "PRESS ENTER TO EXIT" in output:
                process.kill()        # forced shutdown
                break
            if "WARNING: BOLSIG+ convergence failed" in output:
                process.kill()        # forced shutdown
                break
    except:
        pass
    return process

In [27]:
# Error calculation
def cal_error(exp_result):
    # Read a result
    conditions = []
    with open('qt_conditions_list.txt','r') as file:
        for line in file:
            line = line.strip()
            line = line[2:]
            conditions.append(line)
        file.close()

    species = []
    with open('qt_species_list.txt','r') as file:
        for line in file:
            line = line.rstrip()
            line = line[3:]
            species.append(line)
        file.close()

    reactions = []
    reaction_list = pd.read_csv('parameter_set.csv')
    reactions = reaction_list['Reaction'].to_list()
    df_cd = pd.read_csv('qt_conditions.txt', sep=r'\s+', header=0, names=['Time [s]']+conditions)
    df_sp = pd.read_csv('qt_densities.txt', sep=r'\s+', header=0, names=['Time [s]']+species)
    df_rx = pd.read_csv('qt_rates.txt', sep=r'\s+', header=0, names=['Time [s]']+reactions)
    top_rate = df_rx.iloc[:,1:].sum().sort_values(ascending=False)

    CH4 = (df_sp['CH4'] + df_sp['CH4(V13)'] + df_sp['CH4(V24)'])
    C2H2 = (df_sp['C2H2'] + df_sp['C2H2(V13)']+ df_sp['C2H2(V2)']+ df_sp['C2H2(V5)'])
    C2H4 = (df_sp['C2H4'] + df_sp['C2H4(V1)']+ df_sp['C2H4(V2)'])
    C2H6 = (df_sp['C2H6'] + df_sp['C2H6(V13)']+ df_sp['C2H6(V24)'])
    C3H6 = (df_sp['C3H6'] + df_sp['C3H6(V)'])
    C3H8 = (df_sp['C3H8'] + df_sp['C3H8(V1)'] + df_sp['C3H8(V2)'])
    C4H10 = (df_sp['C4H9H'])
    C5H12 = (df_sp['C5H12'])
    H2 = df_sp['H2']
    C = df_sp['C']
    H = df_sp['H']
    CH = df_sp['CH']
    CH2 = df_sp['CH2']
    CH3 = df_sp['CH3']
    C2H3 = df_sp['C2H3']
    C2H5 = df_sp['C2H5']
    C3H5 = df_sp['C3H5']
    C3H7 = df_sp['C3H7']
    C4H9 = df_sp['C4H9']

    exp = exp_result
    
    sim_XCH4 = (CH4.iloc[0] - CH4.iloc[-1])/CH4.iloc[0]*100
    sim_SH2 = 0.5*H2.iloc[-1]/(CH4.iloc[0] - CH4.iloc[-1])*100
    sim_SC2H6 = 2*C2H6.iloc[-1]/(CH4.iloc[0] - CH4.iloc[-1])*100
    sim_SC2H4 = 2*C2H4.iloc[-1]/(CH4.iloc[0] - CH4.iloc[-1])*100
    sim_SC2H2 = 2*C2H2.iloc[-1]/(CH4.iloc[0] - CH4.iloc[-1])*100
    sim_SC3H8 = 3*C3H8.iloc[-1]/(CH4.iloc[0] - CH4.iloc[-1])*100
    sim_SC3H6 = 3*C3H6.iloc[-1]/(CH4.iloc[0] - CH4.iloc[-1])*100
    sim_SC4H10 = 4*C4H10.iloc[-1]/(CH4.iloc[0] - CH4.iloc[-1])*100
    sim_SC5H12 = 5*C5H12.iloc[-1]/(CH4.iloc[0] - CH4.iloc[-1])*100
    sim_SH = 0.25*H.iloc[-1]/(CH4.iloc[0] - CH4.iloc[-1])*100
    sim_SCH = CH.iloc[-1]/(CH4.iloc[0] - CH4.iloc[-1])*100
    sim_SCH2 = CH2.iloc[-1]/(CH4.iloc[0] - CH4.iloc[-1])*100
    sim_SCH3 = CH3.iloc[-1]/(CH4.iloc[0] - CH4.iloc[-1])*100
    sim_SC2H3 = 2*C2H3.iloc[-1]/(CH4.iloc[0] - CH4.iloc[-1])*100
    sim_SC2H5 = 2*C2H5.iloc[-1]/(CH4.iloc[0] - CH4.iloc[-1])*100
    sim_SC3H5 = 3*C3H5.iloc[-1]/(CH4.iloc[0] - CH4.iloc[-1])*100
    sim_SC3H7 = 3*C3H7.iloc[-1]/(CH4.iloc[0] - CH4.iloc[-1])*100
    sim_SC4H9 = 4*C4H9.iloc[-1]/(CH4.iloc[0] - CH4.iloc[-1])*100

    sim = []
    sim.append(sim_XCH4)
    sim.append(sim_SH2)
    sim.append(sim_SC2H6)
    sim.append(sim_SC2H4)
    sim.append(sim_SC2H2)
    sim.append(sim_SC3H8)
    sim.append(sim_SC3H6)
    sim.append(sim_SC4H10+sim_SC5H12)
    sim.append(sim_SH)
    sim.append(sim_SCH)
    sim.append(sim_SCH2)
    sim.append(sim_SCH3)
    sim.append(sim_SC2H3)
    sim.append(sim_SC2H5)
    sim.append(sim_SC3H5)
    sim.append(sim_SC3H7)
    sim.append(sim_SC4H9)

    

    err =  float(np.sum((np.asarray(exp) - np.asarray(sim))**2))
    
    return err, top_rate, sim, df_cd['Time [s]'].iloc[-1], sim_XCH4, sim_SC2H6, sim_SC2H4, sim_SC2H2

    

In [28]:
# initial variable sset sampling: latin_hypercube sampling
def latin_hypercube_sampling(bounds, n_samples):
    return np.random.uniform(
        low = np.array([b[0] for b in bounds]),
        high = np.array([b[1] for b in bounds]),
        size = (n_samples, len(bounds))
    )

In [29]:
# update kinet.inp
def update_kinet(samples,index):
    with open(f'kinet.inp','r') as file:
        lines = file.readlines()
        file.close()
    new_lines=[]
    for i in range(316):
        new_value = 10**samples[i]
        new_value2 = f'{new_value:.4e}'.replace('e-0','e-').replace('e+00','e0').replace('e+0','e+').replace('e','d')
        new_line = f'$ double precision, parameter :: f{i} = {new_value2}\n'
        new_lines.append(new_line)
    new_inp = lines[:18] + new_lines + lines[334:]
    with open(f'./kinet{index}.inp', 'w') as file:
        file.writelines(new_inp)

In [30]:
def GP_Model(X,y):
    avg_pre_Y = sum(y)/len(y)
    X_range = X.max(axis=0) - X.min(axis=0)
    # Gaussian Process Surrogate Model
    constant_kernel = ConstantKernel(constant_value=1.0, constant_value_bounds=(1e-5,1e5))
    RBF_kernel = RBF(1.0, length_scale_bounds = (1e-5,1e5))
    kernel = constant_kernel * RBF_kernel

    gp_model = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10, alpha=1e-6)
    gp_model.fit(X,y)
    return gp_model

In [31]:
# Expected Improvement
def EI(X, gp, y_best, xi):
    mu, sigma = gp.predict(X, return_std=True)
    sigma = sigma.reshape(-1,1)
    improvement = mu - y_best - xi
    Z = improvement / sigma
    ei = improvement * norm.cdf(Z) + sigma * norm.pdf(Z)
    return ei

In [32]:
# Bayesian Optimization for minimization
def BO(gp, bounds, n_iter, X, y, exp_result):
    """Perform Bayesian Optimization using EI for minimization."""
    for i in range(n_iter):
        y_best = np.min(y)  # 현재까지의 최소값
        
        def random_search(bounds, gp, y_best, num_samples=1000, xi=0.01):
            candidate_points = np.random.uniform(
                low = np.array(bounds)[:,0],
                high = np.array(bounds)[:, 1],
                size = (num_samples, len(bounds))
            )

            ei_values = EI(candidate_points, gp, y_best, xi = xi)

            best_index = np.argmax(ei_values)
            return candidate_points[best_index]
        
        x_next = random_search(bounds, gp, y_best)

        # 새로운 관측값 추가 (사용자 정의 시뮬레이션 함수 호출 필요)
        para_index = [12,13,14,24,134,220,34,276,273,223,46,116,226,313,315,22]
        paraset = -16 * np.ones(316)
        paraset[list(range(12))] = 0
        paraset[list(range(90,102))] = 0
        for j in range(len(para_index)):
            paraset[para_index[j]] = x_next[j]
     

        update_kinet(paraset,i+20)
        inp_path = f'kinet{i+20}.inp\n'
        exe_path = 'run_plasRxn_v2.exe'
        run_prep(inp_path)
        compile_zdp(exe_path)
        run_exe(exe_path)
        y_next, _, _, t_time, XCH4, SC2H6, SC2H4, SC2H2 = cal_error(exp_result)  # 사용자의 실제 목적 함수

        # 데이터셋 업데이트
        if t_time > 2.1:
            X = np.vstack([X, x_next])
            y = np.append(y, y_next)
            
            bnds_low = [i[0] for i in bounds]
            bnds_high = [i[1] for i in bounds]

            for j in range(len(bounds)):
                det = x_next[j]*1.5
                if det > bnds_high[j]:
                    bnds_high[j] = det
                elif det < bnds_low[j]:
                    bnds_low[j] = det
                else:
                    pass
            bounds = [[bnds_low[j],bnds_high[j]] for j in range(len(bnds_low))]

            np.savetxt('X_final.csv', X, delimiter=',', fmt='%f')
            np.savetxt('y_final.csv', y, delimiter=',', fmt='%f')
        else:
            X = np.vstack([X, x_next])
            y = np.append(y, 20000)

        
        # Surrogate 모델 업데이트
        gp = GP_Model(X, y)
        clear_output()
        print(f'''현재 iteration: {i}, dataset 크기: {len(X)}, 현재 최소값: {y_best}, 이번 y: {y[-1]}\n 
              전환율: {XCH4}, C2H6: {SC2H6}, C2H4: {SC2H4}, C2H2: {SC2H2}''',)
    return X, y

In [33]:
# Optimization
df_set = pd.read_csv('parameter_set.csv')
para_index = [12,13,14,24,134,220,34,276,273,223,46,116,226,313,315,22]
problem = {
    'num_vars': len(para_index),
    'bounds': [[-1,1]] * len(para_index)
}

initial_samples = latin_hypercube_sampling(problem['bounds'],20)

In [34]:
db_error = []
db_paraset = []
db_toprate = []
db_leng = []
result_list = ['XCH4','SH2','SC2H6','SC2H4','SC2H2','SC3H8','SC3H6','SC4+','SH','SCH','SCH2','SCH3','SC2H3','SC2H5','SC3H5','SC3H7','SC4H9']
exp_result = [16.1,77.03,13,1.535,1.302,3.499,0.468,0.423+0.635,1e-3,1e-3,1e-3,1e-3,1e-3,1e-3,1e-3,1e-3,1e-3]

In [35]:
for i in range(len(initial_samples)):
    inp_path = f'kinet{i}.inp\n'
    exe_path = 'run_plasRxn_v2.exe'

    print(f'run {i}')

    paraset = -16 * np.ones(316)
    paraset[list(range(12))] = 0
    paraset[list(range(90,102))] = 0
    for j in range(len(para_index)):
        paraset[para_index[j]] = initial_samples[i][j]

    update_kinet(paraset,i)
    prep_process = run_prep(inp_path)
    prep_process.wait()
    compile_zdp(exe_path)
    exe_process = run_exe(exe_path)
    exe_process.wait()
    error, top_rate, sim_result, total_time,_,_,_,_ = cal_error(exp_result)
    if float(total_time) > 2.1:
        db_error.append(error)
        db_paraset.append(initial_samples[i])
        db_toprate.append(top_rate)
    clear_output()

In [36]:
with open('db_error.txt', 'w') as file:
    for i in range(len(db_error)):
        file.writelines(str(db_error[i]))
        file.writelines('\n')
    file.close()
with open('db_paraset.txt', 'w') as file:
    for i in range(len(db_paraset)):
        file.writelines(str(db_paraset[i].tolist()))
        file.writelines('\n')
    file.close()

In [37]:
# Load Data
with open('db_error.txt', 'r') as file:
    lines_error = file.readlines()
    file.close()
with open('db_paraset.txt', 'r') as file:
    lines_paraset = file.readlines()
    file.close()
pre_X = np.array([ast.literal_eval(i) for i in lines_paraset])
pre_Y = np.array([float(i[:-1]) for i in lines_error])

In [38]:
# Main Reaction
rank_toprate = []
db_toprate[0].index[0]
for i in range(len(db_toprate)):
    rank_set = []
    for j in range(len(db_toprate[0])):
        rank_set.append(int(df_set.loc[df_set['Reaction'] == db_toprate[i].index[j], 'index'].values[0]))
    rank_toprate.append(rank_set)


In [39]:
with open('rank_toprate.txt', 'w') as file:
    for i in range(len(rank_toprate)):
        file.writelines(str(rank_toprate[i]))
        file.writelines('\n')
    file.close()

In [40]:
pre_X

array([[ 0.05952356, -0.15311981, -0.56265202,  0.21089627,  0.35857878,
         0.60801783,  0.32707047, -0.72922154,  0.95200966,  0.31575423,
         0.2354978 ,  0.81955402, -0.8928639 ,  0.41795424,  0.43543812,
        -0.48743918],
       [ 0.79207736,  0.5571504 ,  0.55065129, -0.32045366,  0.64421748,
        -0.46950403,  0.61638327,  0.13643592, -0.52438181, -0.83957163,
        -0.24076423,  0.42635753,  0.93842324, -0.40900623,  0.82911391,
         0.79702677],
       [ 0.04705247, -0.62300341, -0.89800561,  0.46883491,  0.40102903,
         0.776093  , -0.32102094,  0.23369664, -0.79616005,  0.07563829,
        -0.74531154,  0.73850337,  0.37844658,  0.56254517, -0.07953695,
         0.57092188],
       [-0.5953217 ,  0.33244878, -0.30172234,  0.18627058,  0.04846379,
         0.14676546, -0.96322823, -0.04773465,  0.01987877, -0.95767309,
        -0.16722619,  0.51295978,  0.03233015,  0.11568853,  0.74763098,
         0.24091051],
       [ 0.18770962,  0.68865946, -0

In [41]:
gp_model = GP_Model(pre_X,pre_Y)

c:\Users\USER\miniconda3\envs\plaskinsol\Lib\site-packages\sklearn\gaussian_process\kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


In [ ]:
bounds = [[-1,1]] * len(para_index)
X_final, y_final = BO(gp_model, bounds, 10000,pre_X,pre_Y,exp_result)


현재 iteration: 10, dataset 크기: 21, 현재 최소값: 4811.577688456466, 이번 y: 20000.0
 
              전환율: -4.261414829627826e-06, C2H6: -9.403316429406197e-07, C2H4: -3.380858538666581e-11, C2H2: -4.0264007961446317e-20
check the run of preprocessor


In [ ]:
np.savetxt('X_final.csv', X_final, delimiter=',', fmt='%f')
np.savetxt('y_final.csv', y_final, delimiter=',', fmt='%f')

In [ ]:
pre_Y

array([8.06319318, 8.54267413, 8.00660279, 9.87233875, 8.7643123 ,
       8.89084656])